In [13]:
import cv2 as cv
import dlib
import numpy as np
import pygame
from scipy.spatial import distance as dist
import time

In [14]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C + 1e-7)
    return ear

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[4], mouth[8])
    C = dist.euclidean(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C + 1e-7)
    return mar

In [15]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(".\pre-trained\shape_predictor_68_face_landmarks.dat")
pygame.mixer.init()
pygame.mixer.music.load("alarm.mp3")
player = pygame.mixer.music

In [49]:
EAR_THRESHOLD = 0.22
MAR_THRESHOLD = 0.5

In [50]:
path = r"C:\Users\vkris\Pictures\Camera Roll\WIN_20231122_14_31_04_Pro.jpg"
img = cv.imread(path)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
face = detector(gray)
if (len(face)!=0):
    landmarks = predictor(gray,face[0])
else:
    print("face not found")

In [55]:
left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36,42)], dtype='double')
right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42,48)], dtype='double')
left_eye = eye_aspect_ratio(left_eye)
right_eye = eye_aspect_ratio(right_eye)
if (left_eye<EAR_THRESHOLD and right_eye<EAR_THRESHOLD):
    cv.putText(img,"DROWSY",(10,30),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
else:
    cv.putText(img,"NON DROWSY",(10,30),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
# if (np.all(left_eye) and np.all(right_eye)):
#     ear = (left_eye+right_eye)/2.0
# elif (np.all(left_eye)):
#     ear = left_eye
# elif (np.all(right_eye)):
#     ear = right_eye
# if (ear<EAR_THRESHOLD):
#     cv.putText(img,"DROWSY",(10,30),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
# else:
#     cv.putText(img,"NON DROWSY",(10,30),cv.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
for i in range(68):
    a = landmarks.part(i).x
    b = landmarks.part(i).y
    cv.circle(img,(a,b),1,(0,255,255),1)

cv.imshow('Drowsiness Detection for image',img)
cv.waitKey(0)
cv.destroyAllWindows()